In [1]:
# 구글 드라이브와 연결하기
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!python -m pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 7.9 MB/s eta 0:00:00


In [4]:
%cd /content/gdrive/MyDrive/pytest_img/YOLO/

/content/gdrive/MyDrive/pytest_img/YOLO


In [5]:
!ls

data.yaml    nets	    README.dataset.txt	 result   sample.mp4  train  yolov5
gangnam.jpg  output_frames  README.roboflow.txt  results  test	      valid  yolov5s.pt


In [6]:
from ultralytics import YOLO
model = YOLO("../models/yolov8m-pose.pt")

100%|██████████| 50.8M/50.8M [00:00<00:00, 99.2MB/s]


In [7]:
seconds = 1 # 동영상을 읽을 시간을 초 단위로 설정
video_path = '/content/gdrive/MyDrive/pytest_img/YOLO/walking.mp4' # video path
output_dir = '/content/gdrive/MyDrive/pytest_img/YOLO/results' # 결과 저장 폴더

In [10]:
import torch

def predict(frame, iou=0.7, conf=0.25):
  results = model(
    source = frame,
    device = "0" if torch.cuda.is_available() else "cpu",
    iou = iou, # 바운딩 박스 필터링 신뢰도 기준
    conf = conf, # 모델이 탐지한 객체에 대한 최소 신뢰도 기준
    verbose = False, # 추가 정보 출력 여부
  )
  result = results[0] # 첫 번째 프레임의 이미지 (현재 1개씩의 프레임만 전달됨)
  return result

In [12]:
import cv2
import numpy as np

def draw_keypoints(result, frame):
  # 신체 부위를 따라 키 포인트를 연결하는 리스트
  connections = [
  ([4, 2, 0, 1, 3], (0, 255, 0)), # 얼굴 부위. 초록
  ([10, 8, 6, 5, 7, 9], (255, 0, 0)), # 두 팔. 파랑
  ([6, 12, 11, 5], (255, 0, 255)), # 몸통. 보라
  ([12, 14, 16], (0, 165, 255)), # 오른 다리. 주홍
  ([11, 13, 15], (0, 165, 255)) # 왼 다리. 주홍
  ]
  for kps in result.keypoints: # predict()가 분석한 키 포인트의 원소를 순회
    kps = kps.data.squeeze() # 크기가 1인 불필요한 차원 제거
    nkps = kps.cpu().numpy() # 넘파이 배열로 변환
    for group, color in connections:
      for i in range(len(group) - 1):
        idx1, idx2 = group[i], group[i + 1] # 키 포인트와 이어야 하는 인접 키 포인트
        x1, y1, score1 = nkps[idx1] # x좌표, y좌표, 신뢰도 추출
        x2, y2, score2 = nkps[idx2] # x좌표, y좌표, 신뢰도 추출
        if score1 > 0.5 and score2 > 0.5: # 두 키 포인트의 신뢰도가 0.5보다 큰 경우에만
          point1 = (int(x1), int(y1)) # 키 포인트
          point2 = (int(x2), int(y2)) # 인접한 키 포인트
          cv2.circle(frame, point1, 3, (0, 0, 255), cv2.FILLED) # 각 키 포인트를 빨간색 원으로 그림
          cv2.putText(frame, str(idx1), point1, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1) # 각 키 포인트를 빨간색 텍스트로 표시
          cv2.circle(frame, point2, 3, (0, 0, 255), cv2.FILLED) # 인접 키 포인트
          cv2.putText(frame, str(idx2), point2, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
          cv2.line(frame, point1, point2, color, 2) # 두 키 포인트를 선으로 연결
  return frame # 키 포인트와 연결된 선이 그려진 frame 반환

In [13]:
import cv2
capture = cv2.VideoCapture(video_path)

if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
  capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

fps = capture.get(cv2.CAP_PROP_FPS) # 초당 프레임 수 얻기
total_frames = int(fps * seconds) # seconds 시간 동안의 프레임 수
print('total 프레임:', total_frames) # 출력하게 되는 전체 프레임 수. 25

total 프레임: 25


In [15]:
import os

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

frame_count = 0
while frame_count < total_frames:
  ok, frame = capture.read()
  if not ok:
    print("프레임 읽기에 실패했습니다. 종료.")
    break
  result = predict(frame) # 바운딩 박스와 키포인트 분석
  results = draw_keypoints(result, frame) # 분석된 내용으로 키포인트 연결
  output_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
  cv2.imwrite(output_path, results)
  print("Saved to:", output_path)
  frame_count += 1
  key = cv2.waitKey(10)
  if key == ord('q'):
    print("사용자가 종료를 요청했습니다.")
    break
capture.release()
cv2.destroyAllWindows()


Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0000.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0001.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0002.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0003.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0004.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0005.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0006.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0007.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0008.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0009.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0010.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0011.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0012.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/r